In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import unicodedata
from tkinter import filedialog
import zipfile
import os
import requests
import dask.dataframe as dd

# Função para remover acentos e caracteres especiais
def remove_special_characters(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

# Função para realizar o JOIN entre os DataFrames
def perform_join_and_display_ages():
    # Solicitar a URL do arquivo CSV da Bahia
    url_bahia = bahia_url_entry.get()
    if not url_bahia:
        result_label.config(text="URL da Bahia não fornecida.")
        return

    # Solicitar a URL do arquivo ZIP do Rio de Janeiro
    zip_file_url = rj_zip_url_entry.get()
    if not zip_file_url:
        result_label.config(text="URL do arquivo ZIP do Rio de Janeiro não fornecida.")
        return

    # Caminho para o arquivo ZIP do Rio de Janeiro
    zip_file_path = "C:\\Users\\Eduarda\\OneDrive\\Área de Trabalho\\PIBITI-IFBA-CNPq-2023-2024\\Desafio 1\\ba1.zip"

    # Pasta de destino para extrair os arquivos
    extracted_folder = "C:\\Users\\Eduarda\\OneDrive\\Área de Trabalho\\PIBITI-IFBA-CNPq-2023-2024\\Desafio 1\\extracted_files"

    # Verificar se a pasta de destino existe e, se não existir, criar
    if not os.path.exists(extracted_folder):
        os.makedirs(extracted_folder)

    # Extrair os arquivos do arquivo ZIP
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)

    # Caminho para o arquivo CSV extraído
    csv_file_path = os.path.join(extracted_folder, 'ba1.csv')

    # Especificar tipos de dados para colunas problemáticas
    column_types = {'col12': str, 'col26': int, 'col31': float}

    try:
        # Ler o arquivo CSV do Bahia 2 com especificação de tipos de dados
        df_rj = pd.read_csv(csv_file_path, delimiter=determinar_delimitador(csv_file_path), dtype=column_types, low_memory=False)
    except Exception as e:
        error_message = f"Erro ao ler o arquivo CSV do Bahia 2. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    # Download do arquivo CSV da Bahia
    try:
        response = requests.get(url_bahia)
        with open('bahia.csv', 'wb') as file:
            file.write(response.content)
    except Exception as e:
        error_message = f"Erro ao fazer o download do arquivo CSV da Bahia. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    # Ler o arquivo CSV da Bahia após o download usando Dask
    try:
        df_bahia = dd.read_csv('bahia.csv', delimiter=determinar_delimitador('bahia.csv'))
    except Exception as e:
        error_message = f"Erro ao ler o arquivo CSV da Bahia. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    # Verificar a presença da coluna 'idade' nos DataFrames originais
    if 'idade' not in df_bahia.columns or 'idade' not in df_rj.columns:
        error_message = "A coluna 'idade' não está presente em um dos DataFrames originais."
        result_label.config(text=error_message)
        print(error_message)
        return
    
    # Realizar o merge usando Dask
    join_type = join_type_var.get()  # Tipo de JOIN selecionado (inner, outer, left, right)
    merged_df = dd.merge(df_bahia, df_rj, on='sintomas', how=join_type)
    
    # Exibir a coluna 'idade' após o merge se ela existir
    if 'idade' in merged_df.columns:
        result_label.config(text="Coluna 'idade' após o JOIN:\n" + str(merged_df['idade']))
    else:
        result_label.config(text="A coluna 'idade' não existe no DataFrame resultante.")

# Função para determinar o delimitador do arquivo CSV
def determinar_delimitador(url):
    with open(url, 'r') as file:
        line = file.readline()
        delimiters = [',', ';', '\t']
        for delimiter in delimiters:
            if delimiter in line:
                return delimiter
    return ','  # Delimitador padrão é a vírgula

# Configuração da janela principal
root = tk.Tk()
root.title("Operação de JOIN e Exibição de Idades")

# URL da Bahia
bahia_url_label = ttk.Label(root, text="URL do arquivo CSV da Bahia:")
bahia_url_entry = ttk.Entry(root, width=40)

# URL do arquivo ZIP do Rio de Janeiro
rj_zip_url_label = ttk.Label(root, text="URL do arquivo ZIP do Rio de Janeiro:")
rj_zip_url_entry = ttk.Entry(root, width=40)

# Tipo de JOIN
join_type_var = tk.StringVar()
join_type_label = ttk.Label(root, text="Tipo de JOIN:")
join_type_combobox = ttk.Combobox(root, textvariable=join_type_var, values=["inner", "outer", "left", "right"])

# Botão para realizar a operação de JOIN
join_button = ttk.Button(root, text="Realizar JOIN e Exibir Idades", command=perform_join_and_display_ages)

# Rótulo para exibir a coluna 'IDADES' após o merge
result_label = ttk.Label(root, text="")

# Posicionamento dos widgets
bahia_url_label.grid(row=0, column=0, sticky="w")
bahia_url_entry.grid(row=0, column=1, columnspan=2, sticky="w")
rj_zip_url_label.grid(row=1, column=0, sticky="w")
rj_zip_url_entry.grid(row=1, column=1, columnspan=2, sticky="w")
join_type_label.grid(row=2, column=0, sticky="w")
join_type_combobox.grid(row=2, column=1, columnspan=2, sticky="w")
join_button.grid(row=3, column=0, columnspan=3, pady=10)
result_label.grid(row=4, column=0, columnspan=3, pady=10)

# Iniciar a interface gráfica
root.mainloop()